# ***INSTALLATION DE H2O***

In [1]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 164.6MB 95kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.4-py2.py3-none-any.whl size=164670979 sha256=edbd4759debad6a591a0a83b2ec2972d30aca7e3f84dceb603969e7f21fff2ff
  Stored in directory: /root/.cache/pip/wheels/65/f4/0d/c9bb958d70c2e015c968cb91cbd7f1b486933056d422337d75
Successfully built h2o


# ***IMPORTATION DES LIBRAIRIES***

In [2]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import datetime as dt
import h2o
import xgboost
import imblearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#h2o.cluster().shutdown()

response_column = 'Delay.Indicator'

from google.colab import files
uploaded = files.upload()

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Saving test.csv to test.csv
Saving traffic.csv to traffic.csv
Saving train.csv to train.csv
Saving vancouver.csv to vancouver.csv
Saving victoria.csv to victoria.csv


# ***EXPLORATION DU TRAIN SET***

In [3]:
#train = pd.read_csv(r'C:\Users\BP\Desktop\Kaggle competition\train.csv', sep = ",", header = 0)
train = pd.read_csv("train.csv")
# on supprime les colonnes qui ne se retrouvent pas dans le jeu de données test
train = train.drop(columns=['Status','Full.Date','Trip.Duration'])
train.head()

,Vessel.Name,Scheduled.Departure,Trip,Day,Month,Day.of.Month,Year,Delay.Indicator
0,Spirit of British Columbia,7:00 AM,Tsawwassen to Swartz Bay,Sunday,August,28,2016,0
1,Queen of New Westminster,8:00 AM,Tsawwassen to Swartz Bay,Sunday,August,28,2016,0
2,Spirit of Vancouver Island,9:00 AM,Tsawwassen to Swartz Bay,Sunday,August,28,2016,0
3,Coastal Celebration,10:00 AM,Tsawwassen to Swartz Bay,Sunday,August,28,2016,0
4,Spirit of British Columbia,11:00 AM,Tsawwassen to Swartz Bay,Sunday,August,28,2016,0


In [4]:
train.shape

(49504, 8)

In [5]:
### preprocessing du jeu de données

# On crée une colonne pour les heures et les minutes
train[['Hours','temp']] = train['Scheduled.Departure'].str.split(":",expand=True,) 
train[['Minutes','AM_PM']] = train['temp'].str.split(" ",expand=True,) 
train['Hours'] = train['Hours'].astype(int)
train['Minutes'] = train['Minutes'].astype(int)
train['Day.of.Month'] = train['Day.of.Month'].astype(int)

# on transforme AM-PM en 24h
train['Hours'] = np.where(train['AM_PM']=='PM', train['Hours'] + 12, train['Hours'])
train["Hours"] = train["Hours"].apply(lambda s: 0 if s == 24 else s)
train = train.drop(columns=['temp','AM_PM','Scheduled.Departure'])


### Création variable mois_numérique

train.loc[train['Month'] == 'January', 'Month_num'] = int(1)
train.loc[train['Month'] == 'February', 'Month_num'] = int(2)
train.loc[train['Month'] == 'March', 'Month_num'] = int(3)
train.loc[train['Month'] == 'April', 'Month_num'] = int(4)
train.loc[train['Month'] == 'May', 'Month_num'] = int(5)
train.loc[train['Month'] == 'June', 'Month_num'] = int(6)
train.loc[train['Month'] == 'July', 'Month_num'] = int(7)
train.loc[train['Month'] == 'August', 'Month_num'] = int(8)
train.loc[train['Month'] == 'September', 'Month_num'] = int(9)
train.loc[train['Month'] == 'October', 'Month_num'] = int(10)
train.loc[train['Month'] == 'November', 'Month_num'] = int(11)
train.loc[train['Month'] == 'December', 'Month_num'] = int(12)

train['Month_num'] = train['Month_num'].astype(int)

## pour créer time_ID j'ai besoin de chaines de caractères
train['Year'] = train['Year'].astype(str)
train['Month_num'] = train['Month_num'].astype(str)
train['Day.of.Month'] = train['Day.of.Month'].astype(str)
train['Hours'] = train['Hours'].astype(str)


        
train["Month_num"] = train["Month_num"].apply(lambda s: '0' + s if len(s) < 2 else s)
train["Day.of.Month"] = train["Day.of.Month"].apply(lambda s: '0' + s if len(s) < 2 else s)
train["Hours"] = train["Hours"].apply(lambda s: '0' + s if len(s) < 2 else s)


# création d'une colonne ID basée sur année, mois, jour, heure
train['time_ID'] = train['Year'].astype(str)+train['Month_num'].astype(str)+train['Day.of.Month'].astype(str)+train['Hours'].astype(str)

# convertir chaine caractère en int
train['Year'] = train['Year'].astype(int)
train['Month_num'] = train['Month_num'].astype(int)
train['Day.of.Month'] = train['Day.of.Month'].astype(int)
train['Hours'] = train['Hours'].astype(int)

##transformer Trip en départ et arrivée
    
train['departure'], train['arrival'] = train['Trip'].str.split('to').str

train['departure'] = train.departure.str.strip()

train['arrival'] = train.arrival.str.strip()

train = train.drop('Trip', axis = 1)


train.loc[(train['Month'] == 'June') | (train['Month'] == 'July') | (train['Month'] == 'August'),'season'] = 'summer'
train.loc[(train['Month'] == 'September') | (train['Month'] == 'October') | (train['Month'] == 'November'),'season'] = 'Fall'
train.loc[(train['Month'] == 'December') | (train['Month'] == 'January') | (train['Month'] == 'February'),'season'] = 'Winter'
train.loc[(train['Month'] == 'March') | (train['Month'] == 'April') | (train['Month'] == 'May'),'season'] = 'Spring'


#train['temp'] = train['departure'].apply(lambda x: 1 if x == 'Horseshoe Bay' else 0)




train['busy'] =0

train.loc[(train['Month'] == 'June') | (train['Month'] == 'July') | (train['Month'] == 'August') | (train['Month'] == 'September'),'busy'] = 1
train.loc[((train['Day'] == 'Thursday') & (train['Hours'] >= 13)) |((train['Day'] == 'Friday') & (train['Hours'] >= 13)) |((train['Day'] == 'Saturday') & (train['Hours'] <= 12)) |((train['Day'] == 'Monday') & (train['Hours'] >= 13)) |((train['Day'] == 'Sunday') & (train['Hours'] >= 13)), 'busy'] = 1
train.loc[(((train['Day'] == 'Thursday') & (train['Hours'] >= 13)) |((train['Day'] == 'Friday') & (train['Hours'] >= 13)) |((train['Day'] == 'Saturday') & (train['Hours'] <= 12)) |((train['Day'] == 'Monday') & (train['Hours'] >= 13)) |((train['Day'] == 'Sunday') & (train['Hours'] >= 13))) & ((train['Month'] == 'June') |(train['Month'] == 'July') | (train['Month'] == 'August') |(train['Month'] == 'September')), 'busy'] = 2
  
train = pd.get_dummies(data=train, columns=['season','departure','arrival','Day','Month'])
train = train.drop('Day.of.Month', axis = 1)
train.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


,Vessel.Name,Year,Delay.Indicator,Hours,Minutes,Month_num,time_ID,busy,season_Fall,season_Spring,season_Winter,season_summer,departure_Departure Bay,departure_Duke Point,departure_Horseshoe Bay,departure_Langdale,departure_Swartz Bay,departure_Tsawwassen,arrival_Departure Bay,arrival_Duke Point,arrival_Fulford Harbour (Saltspring Is.),arrival_Horseshoe Bay,arrival_Langdale,arrival_Snug Cove (Bowen Is.),arrival_Southern Gulf Islands,arrival_Swartz Bay,arrival_Tsawwassen,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September
0,Spirit of British Columbia,2016,0,7,0,8,2016082807,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,Queen of New Westminster,2016,0,8,0,8,2016082808,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Spirit of Vancouver Island,2016,0,9,0,8,2016082809,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Coastal Celebration,2016,0,10,0,8,2016082810,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,Spirit of British Columbia,2016,0,11,0,8,2016082811,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
train.columns

Index(['Vessel.Name', 'Year', 'Delay.Indicator', 'Hours', 'Minutes',
       'Month_num', 'time_ID', 'busy', 'season_Fall', 'season_Spring',
       'season_Winter', 'season_summer', 'departure_Departure Bay',
       'departure_Duke Point', 'departure_Horseshoe Bay', 'departure_Langdale',
       'departure_Swartz Bay', 'departure_Tsawwassen', 'arrival_Departure Bay',
       'arrival_Duke Point', 'arrival_Fulford Harbour (Saltspring Is.)',
       'arrival_Horseshoe Bay', 'arrival_Langdale',
       'arrival_Snug Cove (Bowen Is.)', 'arrival_Southern Gulf Islands',
       'arrival_Swartz Bay', 'arrival_Tsawwassen', 'Day_Friday', 'Day_Monday',
       'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday',
       'Day_Wednesday', 'Month_April', 'Month_August', 'Month_December',
       'Month_February', 'Month_January', 'Month_July', 'Month_June',
       'Month_March', 'Month_May', 'Month_November', 'Month_October',
       'Month_September'],
      dtype='object')

# Traffic dataset

In [7]:
#traffic = pd.read_csv(r'C:\Users\BP\Desktop\Kaggle competition\traffic.csv', sep = ",", header = 0)
traffic = pd.read_csv("traffic.csv")
traffic = traffic.dropna()
traffic.head()

,Year,Month,Day,Hour,Minute,Second,Traffic.Ordinal
0,2016,8,28,6.0,53.0,1.0,4
1,2016,8,28,6.0,54.0,0.0,4
2,2016,8,28,6.0,55.0,0.0,5
3,2016,8,28,6.0,56.0,0.0,5
4,2016,8,28,6.0,57.0,1.0,4


In [8]:
traffic['Hour'] = traffic['Hour'].astype(int)
traffic['Minute'] = traffic['Minute'].astype(int)
traffic['Second'] = traffic['Second'].astype(int)

## Aggrégation du dataset selon les secondes

traffic = traffic.groupby(['Year','Month','Day','Hour'], as_index=False).agg({'Traffic.Ordinal':'median'})
traffic['Traffic.Ordinal'] = traffic['Traffic.Ordinal'].astype(int)

## pour créer time_ID j'ai besoin de chaines de caractères
traffic['Year'] = traffic['Year'].astype(str)
traffic['Month'] = traffic['Month'].astype(str)
traffic['Day'] = traffic['Day'].astype(str)
traffic['Hour'] = traffic['Hour'].astype(str)

traffic["Month"] = traffic["Month"].apply(lambda s: '0' + s if len(s) < 2 else s)
traffic["Day"] = traffic["Day"].apply(lambda s: '0' + s if len(s) < 2 else s)
traffic["Hour"] = traffic["Hour"].apply(lambda s: '0' + s if len(s) < 2 else s)

# création d'une colonne ID basée sur année, mois, jour
traffic['time_ID'] = traffic['Year'].astype(str)+traffic['Month'].astype(str)+traffic['Day'].astype(str)+traffic['Hour'].astype(str)

traffic.head()

,Year,Month,Day,Hour,Traffic.Ordinal,time_ID
0,2016,08,28,06,4,2016082806
1,2016,08,28,07,4,2016082807
2,2016,08,28,08,2,2016082808
3,2016,08,28,09,1,2016082809
4,2016,08,28,10,1,2016082810


In [9]:
traffic['Traffic.Ordinal'].value_counts()

1    9686
2    3541
3    1473
5     844
4     324
Name: Traffic.Ordinal, dtype: int64

# Vancouver dataset

In [10]:
#vancouver = pd.read_csv(r'C:\Users\BP\Desktop\Kaggle competition\vancouver.csv', sep = ",", header = 0)
vancouver = pd.read_csv("vancouver.csv")
L_Vancouver = vancouver.columns
for col in L_Vancouver:
    vancouver[col+'_Vancouver'] = vancouver[col]

vancouver = vancouver.drop(columns = L_Vancouver)
vancouver.head()

,Date.Time_Vancouver,Year_Vancouver,Month_Vancouver,Day_Vancouver,Time_Vancouver,Temperature.in.Celsius_Vancouver,Dew.Point.Temperature.in.Celsius_Vancouver,Relative Humidity.in.Percent_Vancouver,Humidex.in.Celsius_Vancouver,Hour_Vancouver
0,2016-08-01 00:00:00,2016,8,1,00:00,14.4,12.3,88.0,NaN,0
1,2016-08-01 01:00:00,2016,8,1,01:00,14.9,13.1,89.0,NaN,1
2,2016-08-01 02:00:00,2016,8,1,02:00,15.1,12.9,87.0,NaN,2
3,2016-08-01 03:00:00,2016,8,1,03:00,15.6,13.1,85.0,NaN,3
4,2016-08-01 04:00:00,2016,8,1,04:00,15.0,12.8,87.0,NaN,4


In [11]:
# Imputation des valeurs manquantes
vancouver['Temperature.in.Celsius_Vancouver'] = vancouver['Temperature.in.Celsius_Vancouver'].fillna(method ='pad')
vancouver['Dew.Point.Temperature.in.Celsius_Vancouver'] = vancouver['Dew.Point.Temperature.in.Celsius_Vancouver'].fillna(method ='pad')
vancouver['Relative Humidity.in.Percent_Vancouver'] = vancouver['Relative Humidity.in.Percent_Vancouver'].fillna(method ='pad')

vancouver['Humidex.in.Celsius_Vancouver'] = vancouver.apply(
    lambda row: row['Temperature.in.Celsius_Vancouver'] + (5/9)*(6.11*np.exp(5417.7530*((1/273.16) - (1/(273.16+row['Dew.Point.Temperature.in.Celsius_Vancouver']))))-10) if np.isnan(row['Humidex.in.Celsius_Vancouver']) else row['Humidex.in.Celsius_Vancouver'],
    axis=1
)

## pour créer time_ID j'ai besoin de chaines de caractères
vancouver['Year_Vancouver'] = vancouver['Year_Vancouver'].astype(str)
vancouver['Month_Vancouver'] = vancouver['Month_Vancouver'].astype(str)
vancouver['Day_Vancouver'] = vancouver['Day_Vancouver'].astype(str)
vancouver['Hour_Vancouver'] = vancouver['Hour_Vancouver'].astype(str)

vancouver["Month_Vancouver"] = vancouver["Month_Vancouver"].apply(lambda s: '0' + s if len(s) < 2 else s)
vancouver["Day_Vancouver"] = vancouver["Day_Vancouver"].apply(lambda s: '0' + s if len(s) < 2 else s)
vancouver["Hour_Vancouver"] = vancouver["Hour_Vancouver"].apply(lambda s: '0' + s if len(s) < 2 else s)


# création d'une colonne ID basée sur année, mois, jour
vancouver['time_ID'] = vancouver['Year_Vancouver'].astype(str)+vancouver['Month_Vancouver'].astype(str)+vancouver['Day_Vancouver'].astype(str)+vancouver['Hour_Vancouver'].astype(str)

# convertir chaine caractère en int
vancouver['Year_Vancouver'] = vancouver['Year_Vancouver'].astype(int)
vancouver['Month_Vancouver'] = vancouver['Month_Vancouver'].astype(int)
vancouver['Day_Vancouver'] = vancouver['Day_Vancouver'].astype(int)
vancouver['Hour_Vancouver'] = vancouver['Hour_Vancouver'].astype(int)


vancouver=vancouver.drop(columns = ['Date.Time_Vancouver','Time_Vancouver'])
vancouver.head()

,Year_Vancouver,Month_Vancouver,Day_Vancouver,Temperature.in.Celsius_Vancouver,Dew.Point.Temperature.in.Celsius_Vancouver,Relative Humidity.in.Percent_Vancouver,Humidex.in.Celsius_Vancouver,Hour_Vancouver,time_ID
0,2016,8,1,14.4,12.3,88.0,16.822847,0,2016080100
1,2016,8,1,14.9,13.1,89.0,17.757445,1,2016080101
2,2016,8,1,15.1,12.9,87.0,17.846856,2,2016080102
3,2016,8,1,15.6,13.1,85.0,18.457445,3,2016080103
4,2016,8,1,15.0,12.8,87.0,17.692051,4,2016080104


# Victoria dataset

In [12]:
#victoria = pd.read_csv(r'C:\Users\BP\Desktop\Kaggle competition\victoria.csv', sep = ",", header = 0)
victoria = pd.read_csv("victoria.csv")
L_victoria = victoria.columns
for col in L_victoria:
    victoria[col+'_victoria'] = victoria[col]

victoria = victoria.drop(columns = L_victoria)
victoria = victoria.drop(columns = ['Date.Time_victoria','Time_victoria','Weather_victoria'])
victoria.head()

,Year_victoria,Month_victoria,Day_victoria,Temperature.in.Celsius_victoria,Dew.Point.Temperature.in.Celsius_victoria,Relative.Humidity.in.Percent_victoria,Wind.Direction.in.Degrees_victoria,Wind.Speed.km.per.h_victoria,Visibility.in.km_victoria,Station.Pressure.in.kPa_victoria,Hour_victoria
0,2016,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2016,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2016,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2016,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2016,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [13]:
## Valeurs manquantes
L = victoria.columns[victoria.isnull().any()]

for col in L:
    victoria[col] = victoria.groupby(['Day_victoria'])[col].fillna(method='ffill')
    victoria[col] = victoria.groupby(['Day_victoria'])[col].fillna(method='bfill')

## Création varaible humidité pour Victoria
victoria['Humidex.in.Celsius_victoria'] = victoria.apply(
    lambda row: row['Temperature.in.Celsius_victoria'] + (5/9)*(6.11*np.exp(5417.7530*((1/273.16) - (1/(273.16+row['Dew.Point.Temperature.in.Celsius_victoria']))))-10),
    axis=1
)

## pour créer time_ID j'ai besoin de chaines de caractères
victoria['Year_victoria'] = victoria['Year_victoria'].astype(str)
victoria['Month_victoria'] = victoria['Month_victoria'].astype(str)
victoria['Day_victoria'] = victoria['Day_victoria'].astype(str)
victoria['Hour_victoria'] = victoria['Hour_victoria'].astype(str)

victoria["Month_victoria"] = victoria["Month_victoria"].apply(lambda s: '0' + s if len(s) < 2 else s)
victoria["Day_victoria"] = victoria["Day_victoria"].apply(lambda s: '0' + s if len(s) < 2 else s)
victoria["Hour_victoria"] = victoria["Hour_victoria"].apply(lambda s: '0' + s if len(s) < 2 else s)


# création d'une colonne ID basée sur année, mois, jour
victoria['time_ID'] = victoria['Year_victoria'].astype(str)+victoria['Month_victoria'].astype(str)+victoria['Day_victoria'].astype(str)+victoria['Hour_victoria'].astype(str)

# convertir chaine caractère en int
victoria['Year_victoria'] = victoria['Year_victoria'].astype(int)
victoria['Month_victoria'] = victoria['Month_victoria'].astype(int)
victoria['Day_victoria'] = victoria['Day_victoria'].astype(int)
victoria['Hour_victoria'] = victoria['Hour_victoria'].astype(int)

victoria.head()

,Year_victoria,Month_victoria,Day_victoria,Temperature.in.Celsius_victoria,Dew.Point.Temperature.in.Celsius_victoria,Relative.Humidity.in.Percent_victoria,Wind.Direction.in.Degrees_victoria,Wind.Speed.km.per.h_victoria,Visibility.in.km_victoria,Station.Pressure.in.kPa_victoria,Hour_victoria,Humidex.in.Celsius_victoria,time_ID
0,2016,8,1,12.5,11.1,91.0,360.0,1.0,24.1,101.81,0,14.308557,2016080100
1,2016,8,1,12.5,11.1,91.0,360.0,1.0,24.1,101.81,1,14.308557,2016080101
2,2016,8,1,12.5,11.1,91.0,360.0,1.0,24.1,101.81,2,14.308557,2016080102
3,2016,8,1,12.5,11.1,91.0,360.0,1.0,24.1,101.81,3,14.308557,2016080103
4,2016,8,1,12.5,11.1,91.0,360.0,1.0,24.1,101.81,4,14.308557,2016080104


# MERGING DATAFRAMES 

In [14]:
train.shape

(49504, 46)

In [15]:
# merging train and traffic
D1 = pd.merge(train, traffic, on='time_ID', how='left')

# merging D1 and vancouver
D2 = pd.merge(D1, vancouver, on='time_ID', how='left')

# merging D2 and victoria
train_final = pd.merge(D2, victoria, on='time_ID', how='left')
train_final['Traffic.Ordinal'] = train_final['departure_Horseshoe Bay'] * train_final['Traffic.Ordinal']
train_final.head()

,Vessel.Name,Year_x,Delay.Indicator,Hours,Minutes,Month_num,time_ID,busy,season_Fall,season_Spring,season_Winter,season_summer,departure_Departure Bay,departure_Duke Point,departure_Horseshoe Bay,departure_Langdale,departure_Swartz Bay,departure_Tsawwassen,arrival_Departure Bay,arrival_Duke Point,arrival_Fulford Harbour (Saltspring Is.),arrival_Horseshoe Bay,arrival_Langdale,arrival_Snug Cove (Bowen Is.),arrival_Southern Gulf Islands,arrival_Swartz Bay,arrival_Tsawwassen,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September,Year_y,Month,Day,Hour,Traffic.Ordinal,Year_Vancouver,Month_Vancouver,Day_Vancouver,Temperature.in.Celsius_Vancouver,Dew.Point.Temperature.in.Celsius_Vancouver,Relative Humidity.in.Percent_Vancouver,Humidex.in.Celsius_Vancouver,Hour_Vancouver,Year_victoria,Month_victoria,Day_victoria,Temperature.in.Celsius_victoria,Dew.Point.Temperature.in.Celsius_victoria,Relative.Humidity.in.Percent_victoria,Wind.Direction.in.Degrees_victoria,Wind.Speed.km.per.h_victoria,Visibility.in.km_victoria,Station.Pressure.in.kPa_victoria,Hour_victoria,Humidex.in.Celsius_victoria
0,Spirit of British Columbia,2016,0,7,0,8,2016082807,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2016,08,28,07,0.0,2016,8,28,15.8,12.4,80.0,18.276050,7,2016,8,28,13.1,11.7,91.0,160.0,8.0,24.1,101.79,7,15.210198
1,Queen of New Westminster,2016,0,8,0,8,2016082808,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2016,08,28,08,0.0,2016,8,28,16.1,12.6,80.0,18.683410,8,2016,8,28,13.7,11.9,89.0,180.0,9.0,24.1,101.83,8,15.913174
2,Spirit of Vancouver Island,2016,0,9,0,8,2016082809,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2016,08,28,09,0.0,2016,8,28,16.1,12.4,79.0,18.576050,9,2016,8,28,14.2,11.7,85.0,150.0,10.0,24.1,101.84,9,16.310198
3,Coastal Celebration,2016,0,10,0,8,2016082810,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2016,08,28,10,0.0,2016,8,28,16.7,12.3,75.0,19.122847,10,2016,8,28,14.6,11.8,83.0,160.0,14.0,24.1,101.88,10,16.761532
4,Spirit of British Columbia,2016,0,11,0,8,2016082811,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2016,08,28,11,0.0,2016,8,28,17.4,11.9,70.0,19.613174,11,2016,8,28,15.2,11.8,80.0,120.0,11.0,24.1,101.87,11,17.361532


In [16]:
train_final['Traffic.Ordinal'].value_counts()

0.0    34688
1.0     8686
2.0     3913
3.0     1262
5.0      626
4.0      243
Name: Traffic.Ordinal, dtype: int64

In [17]:
train_final.columns

Index(['Vessel.Name', 'Year_x', 'Delay.Indicator', 'Hours', 'Minutes',
       'Month_num', 'time_ID', 'busy', 'season_Fall', 'season_Spring',
       'season_Winter', 'season_summer', 'departure_Departure Bay',
       'departure_Duke Point', 'departure_Horseshoe Bay', 'departure_Langdale',
       'departure_Swartz Bay', 'departure_Tsawwassen', 'arrival_Departure Bay',
       'arrival_Duke Point', 'arrival_Fulford Harbour (Saltspring Is.)',
       'arrival_Horseshoe Bay', 'arrival_Langdale',
       'arrival_Snug Cove (Bowen Is.)', 'arrival_Southern Gulf Islands',
       'arrival_Swartz Bay', 'arrival_Tsawwassen', 'Day_Friday', 'Day_Monday',
       'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday',
       'Day_Wednesday', 'Month_April', 'Month_August', 'Month_December',
       'Month_February', 'Month_January', 'Month_July', 'Month_June',
       'Month_March', 'Month_May', 'Month_November', 'Month_October',
       'Month_September', 'Year_y', 'Month', 'Day', 'Hour', 'Traffic.Ord

In [18]:
train_final = train_final.drop(columns=['Traffic.Ordinal','Year_y','Hour','Year_Vancouver','time_ID','Month_Vancouver','Day_Vancouver','Hour_Vancouver','Year_victoria','Month_victoria','Day_victoria','Hour_victoria'])



In [19]:
train_final.columns

Index(['Vessel.Name', 'Year_x', 'Delay.Indicator', 'Hours', 'Minutes',
       'Month_num', 'busy', 'season_Fall', 'season_Spring', 'season_Winter',
       'season_summer', 'departure_Departure Bay', 'departure_Duke Point',
       'departure_Horseshoe Bay', 'departure_Langdale', 'departure_Swartz Bay',
       'departure_Tsawwassen', 'arrival_Departure Bay', 'arrival_Duke Point',
       'arrival_Fulford Harbour (Saltspring Is.)', 'arrival_Horseshoe Bay',
       'arrival_Langdale', 'arrival_Snug Cove (Bowen Is.)',
       'arrival_Southern Gulf Islands', 'arrival_Swartz Bay',
       'arrival_Tsawwassen', 'Day_Friday', 'Day_Monday', 'Day_Saturday',
       'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday',
       'Month_April', 'Month_August', 'Month_December', 'Month_February',
       'Month_January', 'Month_July', 'Month_June', 'Month_March', 'Month_May',
       'Month_November', 'Month_October', 'Month_September', 'Month', 'Day',
       'Temperature.in.Celsius_Vancouver',
     

In [20]:
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

# Les Heures, les minutes, les jours et les mois sont cycliques
train_final['Minutes_sin'] = np.sin(train_final['Minutes']*(2.*np.pi/60))
train_final['Minutes_cos'] = np.cos(train_final['Minutes']*(2.*np.pi/60))

train_final['Hours_sin'] = np.sin(train_final['Hours']*(2.*np.pi/24))
train_final['Hours_cos'] = np.cos(train_final['Hours']*(2.*np.pi/24))


train_final = train_final.drop(columns=['Year_x','Hours','Minutes','Month_num'])


train_final.head()

,Vessel.Name,Delay.Indicator,busy,season_Fall,season_Spring,season_Winter,season_summer,departure_Departure Bay,departure_Duke Point,departure_Horseshoe Bay,departure_Langdale,departure_Swartz Bay,departure_Tsawwassen,arrival_Departure Bay,arrival_Duke Point,arrival_Fulford Harbour (Saltspring Is.),arrival_Horseshoe Bay,arrival_Langdale,arrival_Snug Cove (Bowen Is.),arrival_Southern Gulf Islands,arrival_Swartz Bay,arrival_Tsawwassen,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September,Month,Day,Temperature.in.Celsius_Vancouver,Dew.Point.Temperature.in.Celsius_Vancouver,Relative Humidity.in.Percent_Vancouver,Humidex.in.Celsius_Vancouver,Temperature.in.Celsius_victoria,Dew.Point.Temperature.in.Celsius_victoria,Relative.Humidity.in.Percent_victoria,Wind.Direction.in.Degrees_victoria,Wind.Speed.km.per.h_victoria,Visibility.in.km_victoria,Station.Pressure.in.kPa_victoria,Humidex.in.Celsius_victoria,Minutes_sin,Minutes_cos,Hours_sin,Hours_cos
0,Spirit of British Columbia,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,08,28,15.8,12.4,80.0,18.276050,13.1,11.7,91.0,160.0,8.0,24.1,101.79,15.210198,0.0,1.0,0.965926,-0.258819
1,Queen of New Westminster,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,08,28,16.1,12.6,80.0,18.683410,13.7,11.9,89.0,180.0,9.0,24.1,101.83,15.913174,0.0,1.0,0.866025,-0.500000
2,Spirit of Vancouver Island,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,08,28,16.1,12.4,79.0,18.576050,14.2,11.7,85.0,150.0,10.0,24.1,101.84,16.310198,0.0,1.0,0.707107,-0.707107
3,Coastal Celebration,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,08,28,16.7,12.3,75.0,19.122847,14.6,11.8,83.0,160.0,14.0,24.1,101.88,16.761532,0.0,1.0,0.500000,-0.866025
4,Spirit of British Columbia,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,08,28,17.4,11.9,70.0,19.613174,15.2,11.8,80.0,120.0,11.0,24.1,101.87,17.361532,0.0,1.0,0.258819,-0.965926


In [21]:
train_final.dtypes

Vessel.Name                                    object
Delay.Indicator                                 int64
busy                                            int64
season_Fall                                     uint8
season_Spring                                   uint8
season_Winter                                   uint8
season_summer                                   uint8
departure_Departure Bay                         uint8
departure_Duke Point                            uint8
departure_Horseshoe Bay                         uint8
departure_Langdale                              uint8
departure_Swartz Bay                            uint8
departure_Tsawwassen                            uint8
arrival_Departure Bay                           uint8
arrival_Duke Point                              uint8
arrival_Fulford Harbour (Saltspring Is.)        uint8
arrival_Horseshoe Bay                           uint8
arrival_Langdale                                uint8
arrival_Snug Cove (Bowen Is.

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split

X = train_final.drop(columns=[response_column])
y = train_final.loc[:,[response_column]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,stratify=y, random_state=42)

# on recré le pandas dataframe : train
train_set = pd.concat([X_train,y_train], axis=1)
# on crée dataset test
test_set = pd.concat([X_test,y_test], axis=1)


h2o.init()

# Conversion en h2o df
train_h2o = h2o.H2OFrame(train_set)
test_h2o = h2o.H2OFrame(test_set)

train_h2o[response_column] = train_h2o[response_column].asfactor()
test_h2o[response_column] = test_h2o[response_column].asfactor()

#train_h2o['Day.of.Month'] = train_h2o['Day.of.Month'].asfactor()
#test_h2o['Day.of.Month'] = test_h2o['Day.of.Month'].asfactor()

#train_h2o['Year_x'] = train_h2o['Year_x'].asfactor()
#test_h2o['Year_x'] = test_h2o['Year_x'].asfactor()


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpoyrfa_dd
  JVM stdout: /tmp/tmpoyrfa_dd/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpoyrfa_dd/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 3 days
H2O_cluster_name:,H2O_from_python_unknownUser_adt5u7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
train_h2o.describe()

Rows:37128
Cols:59




,Vessel.Name,busy,season_Fall,season_Spring,season_Winter,season_summer,departure_Departure Bay,departure_Duke Point,departure_Horseshoe Bay,departure_Langdale,departure_Swartz Bay,departure_Tsawwassen,arrival_Departure Bay,arrival_Duke Point,arrival_Fulford Harbour (Saltspring Is.),arrival_Horseshoe Bay,arrival_Langdale,arrival_Snug Cove (Bowen Is.),arrival_Southern Gulf Islands,arrival_Swartz Bay,arrival_Tsawwassen,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September,Month,Day,Temperature.in.Celsius_Vancouver,Dew.Point.Temperature.in.Celsius_Vancouver,Relative Humidity.in.Percent_Vancouver,Humidex.in.Celsius_Vancouver,Temperature.in.Celsius_victoria,Dew.Point.Temperature.in.Celsius_victoria,Relative.Humidity.in.Percent_victoria,Wind.Direction.in.Degrees_victoria,Wind.Speed.km.per.h_victoria,Visibility.in.km_victoria,Station.Pressure.in.kPa_victoria,Humidex.in.Celsius_victoria,Minutes_sin,Minutes_cos,Hours_sin,Hours_cos,Delay.Indicator
type,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,real,real,real,int,int,int,real,real,real,real,real,real,real,enum
mins,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-5.0,-14.4,26.0,-8.57136833955947,-3.7,-15.8,26.0,0.0,0.0,0.2,98.02,-7.162377706896365,-1.0,-1.0,-1.0,-0.9659258262890684,
mean,,0.7728668390433094,0.37311463046757165,0.19540508511096746,0.1907455289808231,0.2407347554406378,0.07735401853048912,0.06816957552251669,0.29624542124542125,0.09036306830424477,0.2655408317173023,0.20232708468002586,0.07676147382029735,0.06997414350355527,0.07258672699849171,0.16771708683473388,0.08882783882783883,0.13065610859728508,0.11770092652445593,0.10350678733031674,0.1722689075630252,0.15346907993966818,0.1466009480715363,0.13577354018530488,0.14237233354880413,0.14377289377289376,0.13935574229691877,0.13865546218487396,0.06227106227106227,0.09127881921999569,0.06348308554190907,0.06377935789700495,0.06348308554190907,0.08061301443654385,0.06884292178409826,0.06822344322344322,0.06491057961646197,0.11220642102995045,0.12718164188752423,0.13372656755009696,7.148885651071197,15.74108251038802,11.84166666666661,7.829422538246069,78.24170437405714,12.50980681419789,10.9324472096531,6.799881491057963,76.68172268907567,207.74725274725333,11.58907024348198,22.256693061840163,101.54910929756498,11.138622888953384,0.00046327266437096247,0.1970682629215796,-0.15190742811422395,-0.20051445938889148,
maxs,,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12.0,31.0,30.0,18.3,99.0,34.0,27.3,15.4,99.0,360.0,61.0,80.5,103.84,27.782583675253495,1.0,1.0,1.0,1.0,
sigma,,0.6954710380843165,0.48363871122843327,0.3965175563956316,0.39289417127367277,0.42753532732003513,0.26715594075079035,0.2520400671456377,0.45660670939726533,0.2867050717164916,0.44162671050204316,0.4017401927597211,0.266216188064762,0.2551068709118595,0.25946041555710614,0.3736198942290874,0.2844989172088536,0.3370284101522093,0.32225799525522336,0.30462375424536386,0.37761908282558054,0.36044392168985506,0.3537124253640047,0.34255254553042397,0.3494363189000824,0.3508640256508599,0.3463220319349207,0.3455912929906381,0.24165047046414687,0.28800908062762537,0.2438331083607849,0.24436276252441774,0.24383310836078492,0.27224355379722326,0.25319024567882653,0.25213234069109886,0.2463713277221148,0.3156244971926103,0.3331808244280589,0.34036288411672605,3.3179646428144034,8.845488428864348,6.341992294291478,5.256902670842993,14.671369720487549,8.148377810565464,4.756767844629416,4.989476304930929,12.836260065

In [25]:
from h2o.automl import H2OAutoML

# Identify predictors and response
x = train_h2o.columns
y = response_column
x.remove(y)


# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(nfolds = 5, balance_classes = True, seed=1234,max_models = 20)#, max_runtime_secs = 300)
aml.train(x=x, y=y, training_frame=train_h2o)

# View the AutoML Leaderboard
aml.leaderboard

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20210304_150433,0.876399,0.316284,0.678703,0.2215,0.308052,0.094896
StackedEnsemble_BestOfFamily_AutoML_20210304_150433,0.874609,0.317795,0.673844,0.225698,0.308848,0.0953869
GBM_5_AutoML_20210304_150433,0.873277,0.326752,0.668001,0.223038,0.315474,0.0995237
GBM_4_AutoML_20210304_150433,0.871414,0.33653,0.667199,0.229243,0.318654,0.10154
GBM_3_AutoML_20210304_150433,0.868273,0.331859,0.652612,0.229102,0.318853,0.101667
GBM_2_AutoML_20210304_150433,0.866766,0.331748,0.647725,0.235164,0.319294,0.101949
GBM_grid__1_AutoML_20210304_150433_model_2,0.865652,0.366611,0.652773,0.223143,0.328771,0.10809
DRF_1_AutoML_20210304_150433,0.862264,0.383839,0.638835,0.229945,0.340681,0.116063
GBM_1_AutoML_20210304_150433,0.861691,0.33538,0.634391,0.234627,0.321782,0.103543
XGBoost_grid__1_AutoML_20210304_150433_model_3,0.856259,0.333272,0.62952,0.24534,0.318459,0.101416


In [26]:
# predictions
predictions = aml.leader.model_performance(test_data=test_h2o)
auc_score = predictions.auc()
print('auc_score :',auc_score)

auc_score : 0.885584372805759


# PREDICTIONS ON TEST FRAME

In [30]:
#test = pd.read_csv(r'C:\Users\BP\Desktop\Kaggle competition\test.csv', sep = ",", header = 0)
test = pd.read_csv("test.csv")
test = test.drop(columns=['Full.Date'])
test.head()

,ID,Vessel.Name,Scheduled.Departure,Trip,Day,Month,Day.of.Month,Year
0,1,Queen of Coquitlam,5:15 AM,Duke Point to Tsawwassen,Monday,November,27,2017
1,2,Queen of New Westminster,7:45 AM,Duke Point to Tsawwassen,Monday,November,27,2017
2,3,Queen of Coquitlam,10:15 AM,Duke Point to Tsawwassen,Monday,November,27,2017
3,4,Queen of New Westminster,12:45 PM,Duke Point to Tsawwassen,Monday,November,27,2017
4,5,Queen of Coquitlam,3:15 PM,Duke Point to Tsawwassen,Monday,November,27,2017


In [31]:
### preprocessing du jeu de données

# On crée une colonne pour les heures et les minutes
test[['Hours','temp']] = test['Scheduled.Departure'].str.split(":",expand=True,) 
test[['Minutes','AM_PM']] = test['temp'].str.split(" ",expand=True,) 
test['Hours'] = test['Hours'].astype(int)
test['Minutes'] = test['Minutes'].astype(int)
test['Day.of.Month'] = test['Day.of.Month'].astype(int)

# on transforme AM-PM en 24h
test['Hours'] = np.where(test['AM_PM']=='PM', test['Hours'] + 12, test['Hours'])
test["Hours"] = test["Hours"].apply(lambda s: 0 if s == 24 else s)
test = test.drop(columns=['temp','AM_PM','Scheduled.Departure'])


### Création variable mois_numérique

test.loc[test['Month'] == 'January', 'Month_num'] = int(1)
test.loc[test['Month'] == 'February', 'Month_num'] = int(2)
test.loc[test['Month'] == 'March', 'Month_num'] = int(3)
test.loc[test['Month'] == 'April', 'Month_num'] = int(4)
test.loc[test['Month'] == 'May', 'Month_num'] = int(5)
test.loc[test['Month'] == 'June', 'Month_num'] = int(6)
test.loc[test['Month'] == 'July', 'Month_num'] = int(7)
test.loc[test['Month'] == 'August', 'Month_num'] = int(8)
test.loc[test['Month'] == 'September', 'Month_num'] = int(9)
test.loc[test['Month'] == 'October', 'Month_num'] = int(10)
test.loc[test['Month'] == 'November', 'Month_num'] = int(11)
test.loc[test['Month'] == 'December', 'Month_num'] = int(12)

test['Month_num'] = test['Month_num'].astype(int)

## pour créer time_ID j'ai besoin de chaines de caractères
test['Year'] = test['Year'].astype(str)
test['Month_num'] = test['Month_num'].astype(str)
test['Day.of.Month'] = test['Day.of.Month'].astype(str)
test['Hours'] = test['Hours'].astype(str)


        
test["Month_num"] = test["Month_num"].apply(lambda s: '0' + s if len(s) < 2 else s)
test["Day.of.Month"] = test["Day.of.Month"].apply(lambda s: '0' + s if len(s) < 2 else s)
test["Hours"] = test["Hours"].apply(lambda s: '0' + s if len(s) < 2 else s)


# création d'une colonne ID basée sur année, mois, jour, heure
test['time_ID'] = test['Year'].astype(str)+test['Month_num'].astype(str)+test['Day.of.Month'].astype(str)+test['Hours'].astype(str)

# convertir chaine caractère en int
test['Year'] = test['Year'].astype(int)
test['Month_num'] = test['Month_num'].astype(int)
test['Day.of.Month'] = test['Day.of.Month'].astype(int)
test['Hours'] = test['Hours'].astype(int)

##transformer Trip en départ et arrivée
    
test['departure'], test['arrival'] = test['Trip'].str.split('to').str

test['departure'] = test.departure.str.strip()

test['arrival'] = test.arrival.str.strip()

test = test.drop('Trip', axis = 1)


test['temp'] = test['departure'].apply(lambda x: 1 if x == 'Horseshoe Bay' else 0)



test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


,ID,Vessel.Name,Day,Month,Day.of.Month,Year,Hours,Minutes,Month_num,time_ID,departure,arrival,temp
0,1,Queen of Coquitlam,Monday,November,27,2017,5,15,11,2017112705,Duke Point,Tsawwassen,0
1,2,Queen of New Westminster,Monday,November,27,2017,7,45,11,2017112707,Duke Point,Tsawwassen,0
2,3,Queen of Coquitlam,Monday,November,27,2017,10,15,11,2017112710,Duke Point,Tsawwassen,0
3,4,Queen of New Westminster,Monday,November,27,2017,0,45,11,2017112700,Duke Point,Tsawwassen,0
4,5,Queen of Coquitlam,Monday,November,27,2017,15,15,11,2017112715,Duke Point,Tsawwassen,0


In [32]:
# merging test and traffic
T1 = pd.merge(test, traffic, on='time_ID', how='left')

# merging D1 and vancouver
T2 = pd.merge(T1, vancouver, on='time_ID', how='left')

# merging D2 and victoria
test_final = pd.merge(T2, victoria, on='time_ID', how='left')
test_final['Traffic.Ordinal'] = test_final['temp'] * test_final['Traffic.Ordinal']
test_final = test_final.drop(columns=['temp','Year_y','Month_y','Day_y','Hour','Year_Vancouver','time_ID','Month_Vancouver','Day_Vancouver','Hour_Vancouver','Year_victoria','Month_victoria','Day_victoria','Hour_victoria'])


# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

# Fit the encoder to the pandas column
test_final['Day_label'] = le.fit_transform(test_final['Day_x'])
test_final['Month_label'] = le.fit_transform(test_final['Month_x'])

# Les Heures, les minutes, les jours et les mois sont cycliques
test_final['Minutes_sin'] = np.sin(test_final['Minutes']*(2.*np.pi/60))
test_final['Minutes_cos'] = np.cos(test_final['Minutes']*(2.*np.pi/60))

test_final['Hours_sin'] = np.sin(test_final['Hours']*(2.*np.pi/24))
test_final['Hours_cos'] = np.cos(test_final['Hours']*(2.*np.pi/24))

test_final['Day_label_sin'] = np.sin(test_final['Day_label']*(2.*np.pi/7))
test_final['Day_label_cos'] = np.cos(test_final['Day_label']*(2.*np.pi/7))

test_final['Month_label_sin'] = np.sin(test_final['Month_label']*(2.*np.pi/12))
test_final['Month_label_cos'] = np.cos(test_final['Month_label']*(2.*np.pi/12))

test_final = test_final.drop(columns=['Day_label','Month_label','Hours','Minutes','Month_num'])


test_final.dtypes

ID                                              int64
Vessel.Name                                    object
Day_x                                          object
Month_x                                        object
Day.of.Month                                    int64
Year_x                                          int64
departure                                      object
arrival                                        object
Traffic.Ordinal                                 int64
Temperature.in.Celsius_Vancouver              float64
Dew.Point.Temperature.in.Celsius_Vancouver    float64
Relative Humidity.in.Percent_Vancouver        float64
Humidex.in.Celsius_Vancouver                  float64
Temperature.in.Celsius_victoria               float64
Dew.Point.Temperature.in.Celsius_victoria     float64
Relative.Humidity.in.Percent_victoria         float64
Wind.Direction.in.Degrees_victoria            float64
Wind.Speed.km.per.h_victoria                  float64
Visibility.in.km_victoria   

In [33]:

test_true_h2o = h2o.H2OFrame(test_final)

test_true_h2o['Day.of.Month'] = test_true_h2o['Day.of.Month'].asfactor()
test_true_h2o['Year_x'] = test_true_h2o['Year_x'].asfactor()


preds = aml.leader.predict(test_true_h2o)
pred_df = preds.as_data_frame()
pred_df.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


/usr/local/lib/python3.7/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'busy': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'season_Fall': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'season_Spring': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'season_Winter': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'season_summer': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'departure_Depart

,predict,p0,p1
0,0,0.944532,0.055468
1,0,0.954859,0.045141
2,0,0.926618,0.073382
3,0,0.938834,0.061166
4,0,0.906972,0.093028


In [ ]:
#pred_df["predict"].value_counts()

### creating new dataframe with predictions

export = pd.DataFrame(test['ID'])
export["Delay.Indicator"]= pred_df['p1']




In [ ]:
#export.head()

export = export.set_index('ID')

export.to_csv('attempt_1.csv', sep = ',')

In [ ]:
export['ID']= export[0]

export_final = pd.DataFrame()
export_final['ID']= export['ID']
export_final['Predicted']=export['Predicted']

export_final = export_final.set_index('ID')

export_final.to_csv('attempt_1.csv', sep = ',')